## chapter 7 앙상블 학습과 랜덤포레스트

- 앙상블: 예측기(분류나 회귀모델)로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있는데 일련의 예측기를 뜻하며 이를 앙상블 학습이라고 하고 앙상블 학습 알고리즘을 앙상블 방법이라고 함.

- 랜덤포레스트: 결정트리의 앙상블

## 7.1. 투표 기반 분류기
- 더 좋은 분류기를 만드는 간단한 방법은 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것으로 다수결 투표로 정해지는 분류기를 직접 투표 분류기라고 함.
- 분류기가 약한 학습기(랜덤 추측보다 조금 더 높은 성능을 내는 분류기)일지라도 충분하게 많고 다양하다면 앙상블은 (높은 정확도를 내는)강한 학습기가 될 수 있음.

- 여러 분류기를 조합하여 투표 기반 분류기를 만들고 훈련시키는 코드임.


In [23]:
import warnings

warnings.filterwarnings('ignore')

# import package

import numpy as np

import os

#5장에서 소개한 moons dataset 불러오기

from sklearn.datasets import make_moons

from sklearn.model_selection import train_test_split

X,y = make_moons(n_samples=100, noise=0.15)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [24]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()


voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [25]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_test))

LogisticRegression 1.0
RandomForestClassifier 1.0
SVC 1.0
VotingClassifier 1.0


## 7.2. 배깅과 페이스팅

- 배깅: 훈련 세트에서 중복을 허용하여 샘플링하는 방식
- 페이스팅: 중복을 허용하지 않고 샘플링하는 방식
- 모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측을 만들며, 수집 함수는 전형적인 분류일 때는 통계적 최빈값(직접 투표 분류기처럼 가장 많은 예측 결과)이고 회귀에 대해서 평균을 계산

## 7.2.1 사이킷런과 배깅의 페이스팅
- 사이킷런은 배깅과 페이스팅을 위해 간편한 api로 구성된 BaggingClassifier를 제공함

In [26]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bag_clf = BaggingClassifier(
DecisionTreeClassifier(), n_estimators=500,
max_samples=50, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

## 7.2.2. obb 평가
- 배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플리되고 어떤 것은 전혀 선택되지 않을 수 있음.
- BaggingClassifier는 기본값으로 중복을 허용하여(bootstrap=True) 훈련 세트의 크기만큼 m개 샘플을 선택
- 평균적으로 각 예측기에 훈련 샘플의 63% 정도만 샘플링 되고 나머지 선택되지 않은 훈련 샘플의 나머지 37%를 obb 샘플이라고 부름


In [27]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True
)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9375

In [28]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.95

In [29]:
bag_clf.oob_decision_function_

array([[1.        , 0.        ],
       [1.        , 0.        ],
       [0.98816568, 0.01183432],
       [0.00564972, 0.99435028],
       [0.        , 1.        ],
       [0.1319797 , 0.8680203 ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.04663212, 0.95336788],
       [0.77540107, 0.22459893],
       [0.61413043, 0.38586957],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.83684211, 0.16315789],
       [0.97959184, 0.02040816],
       [1.        , 0.        ],
       [0.01176471, 0.98823529],
       [0.26666667, 0.73333333],
       [1.        , 0.        ],
       [0.00526316, 0.99473684],
       [0.        , 1.        ],
       [0.93922652, 0.06077348],
       [0.06349206, 0.93650794],
       [0.        , 1.        ],
       [0.9039548 , 0.0960452 ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.

## 7.2.2. oob 평가
- BaggingClassfieir은 기본값으로 중복을 허용하여(bootstrap=True) 훈련 세트의 크기반큼인 m개 샘플을 선택
- 이는 평균적으로 각 예측기에 훈련 샘플의 63% 정도만 샘플링된다는 것을 의미
- 예측기가 훈련되는 동안에는 oob 샘플을 사용하지 않으므로 별도의 검증 세트를 사용하지 않고 oob 샘플을 사용해 평가

In [31]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9375

In [32]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.95

In [33]:
bag_clf.oob_decision_function_

array([[0.98342541, 0.01657459],
       [1.        , 0.        ],
       [0.98809524, 0.01190476],
       [0.05825243, 0.94174757],
       [0.        , 1.        ],
       [0.12690355, 0.87309645],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.03723404, 0.96276596],
       [0.83243243, 0.16756757],
       [0.70414201, 0.29585799],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.90555556, 0.09444444],
       [0.99484536, 0.00515464],
       [1.        , 0.        ],
       [0.01621622, 0.98378378],
       [0.3038674 , 0.6961326 ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.92307692, 0.07692308],
       [0.07954545, 0.92045455],
       [0.        , 1.        ],
       [0.8872549 , 0.1127451 ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.98895028, 0.01104972],
       [0.        , 1.        ],
       [0.

## 7.3. 랜덤 패치와 랜덤 서브스페이스
- 훈련 특성과 샘플을 모두 샘플링하는 것을 랜덤 패치 방식이라하고 훈련 샘플을 모두 사용하고 (bootstrap=False이고 max_samples=1,0로 설정) 특성은 샘플링하는 것을 랜덤 스페이스 방식이라고 함.

## 7.4. 랜덤 포레스트
- 랜덤 포레스트는 일반적으로 배깅 방법을 적용한 결정 트리의 앙상블

In [34]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)

In [35]:
# 트리를 다양하게 만들고 편향을 손해보는 대신 분산을 낮추어 전체적으로 더 훌륭한 모델을 만들어냄.
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features='auto', max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

## 7.4.1. 엑스트라 트리

- 트리를 더욱 무작위하게 만들기 위해 최적의 임곗값을 찾는 대신 후보 특성을 사용해 무작위로 분할한 다음 그 중에서 최상의 분할을 선택
- 익스트림 랜덤 트리, 앙상블: 극단적으로 무작위한 트리의 랜덤 포레스트라 불림
- 사이킷런의 ExtraTreeClassifier을 사용

## 7.4.2 특성 중요도
- 랜덤포레스트의 장점은 특성의 상대적 중요도를 측정하기 쉬움


In [36]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.08423499974833865
sepal width (cm) 0.024654258317504507
petal length (cm) 0.4394344273818735
petal width (cm) 0.45167631455228324


## 7.5. 부스팅
- 부스팅은 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법을 말함.
- 에이다부스트, 그레이디언트 부스팅이 가장 인기있음.